# Accessing LMEC Collections via IIIF API

This notebook provides some tips for using Digital Commonwealth's IIIF API to query the LMEC collections portal and programmatically retrieve metadata about collections items.

### Understanding the IIIF API

Through the BPL/Digital Commonwealth, all of LMEC's maps are compliant with the International Image Interoperability Framework (IIIF). This means you can use IIIF APIs to retrieve Image and Presentation responses for any LMEC collection item.

#### Image API

A **Image API** request can return either *image metadata* or *a static image*.

Let's say we want to request metadata and an image for this recently-accessioned [map of summer resorts along the Boston & Maine Railroad](https://collections.leventhalmap.org/search/commonwealth:g158f6689).

The LMEC's API syntax for requesting image metadata is `BASE_URL` + `IMAGE_ID` + `/info.json`:

    # base URL
    https://iiif.digitalcommonwealth.org/iiif/2/

    # image information request
    https://iiif.digitalcommonwealth.org/iiif/2/IMAGE_ID/info.json

The image ID can be found by parsing the JSON data from any old collections item:


In [1]:
import json
import requests

item = requests.get("https://collections.leventhalmap.org/search/commonwealth:g158f6689.json")
print(item.json()['response']['document']['exemplary_image_ssi'])

commonwealth:7w62hz17g


We can append this image ID to the base URL to retrieve image metadata:

In [17]:
base = "https://iiif.digitalcommonwealth.org/iiif/2/"
imageID = "commonwealth:7w62hz17g"

imageInfo = requests.get(base+imageID)

print(json.dumps(imageInfo.json(), indent=2))

{
  "@context": "http://iiif.io/api/image/2/context.json",
  "@id": "https://iiif.digitalcommonwealth.org/iiif/2/commonwealth:7w62hz17g",
  "protocol": "http://iiif.io/api/image",
  "width": 7938,
  "height": 7111,
  "sizes": [
    {
      "width": 124,
      "height": 111
    },
    {
      "width": 248,
      "height": 222
    },
    {
      "width": 496,
      "height": 444
    },
    {
      "width": 992,
      "height": 889
    },
    {
      "width": 1985,
      "height": 1778
    },
    {
      "width": 3969,
      "height": 3556
    },
    {
      "width": 7938,
      "height": 7111
    }
  ],
  "tiles": [
    {
      "width": 1024,
      "height": 1024,
      "scaleFactors": [
        1,
        2,
        4,
        8,
        16,
        32,
        64
      ]
    }
  ],
  "profile": [
    "http://iiif.io/api/image/2/level2.json",
    {
      "formats": [
        "tif",
        "jpg",
        "gif",
        "png"
      ],
      "qualities": [
        "bitonal",
        "defa

The following syntax will return a static image:

    # full image as JPEG
    https://iiif.digitalcommonwealth.org/iiif/2/IMAGE_ID/full/full/0/default.jpg

And accessing it is as easy as combining a few sets of strings:

In [21]:
iiifSpec = "/full/full/0/default.jpg"

imageStatic = (base+imageID+iiifSpec)
print(imageStatic)

https://iiif.digitalcommonwealth.org/iiif/2/commonwealth:7w62hz17g/full/full/0/default.jpg


By tweaking the `iiifSpec` variable, you can easily manipulate the image, adding parameters for size, rotation, quality, format, and more. See the [IIIF docs](https://iiif.io/api/image/3.0/) for more information.

You can also retrieve image data in greater bulk with a `for` loop or a data frame. For example, the following search query returns 157 pictorial maps in Massachusetts:

    https://collections.leventhalmap.org/search?f%5Bsubject_geographic_sim%5D%5B%5D=Massachusetts&q=pictorial

By looping through this query, we can retrieve IIIF image metadata and URLs in bulk.

For the sake of this example, I've limited the results to 50.

In [43]:
import pandas as pd

# don't forget to append
# `.json` after `search` in the URL!

data = requests.get("https://collections.leventhalmap.org/search.json?f%5Bsubject_geographic_sim%5D%5B%5D=Massachusetts&per_page=50&q=pictorial")

# parse the response using the JSON API
# and view it in a pandas data frame

df = pd.DataFrame(data.json()['response']['docs'])
fields = ['id', 'exemplary_image_ssi']
newFieldNames = {'id':'commonwealth_id', 'exemplary_image_ssi':'iiif_id'}
df_fltr = pd.DataFrame(df[fields])
df_fltr.rename(columns = newFieldNames, inplace = True)
df_fltr


,commonwealth_id,iiif_id
0,commonwealth:0r96fm77v,commonwealth:0r96fm784
1,commonwealth:0r96fm79d,commonwealth:0r96fm805
2,commonwealth:0r96fm81f,commonwealth:0r96fm82q
3,commonwealth:0r96fm73r,commonwealth:0r96fm741
4,commonwealth:hq37z388t,commonwealth:9880z8137
5,commonwealth:bz60gc14x,commonwealth:3f464n44x
6,commonwealth:kp78jz84c,commonwealth:76539h830
7,commonwealth:g732gt059,commonwealth:s1787416m
8,commonwealth:3f4638525,commonwealth:3f463853f
9,commonwealth:3f463850m,commonwealth:3f463851w


Let's say we want to create a list that contains static image URLs for these 50 maps. We could loop through the `iiif_id` column and use the image API syntax to generate a list of image URLs:

In [45]:
for a in df_fltr['iiif_id']:
    a = (base+a+iiifSpec)
    print(a)

https://iiif.digitalcommonwealth.org/iiif/2/commonwealth:0r96fm784/full/full/0/default.jpg
https://iiif.digitalcommonwealth.org/iiif/2/commonwealth:0r96fm805/full/full/0/default.jpg
https://iiif.digitalcommonwealth.org/iiif/2/commonwealth:0r96fm82q/full/full/0/default.jpg
https://iiif.digitalcommonwealth.org/iiif/2/commonwealth:0r96fm741/full/full/0/default.jpg
https://iiif.digitalcommonwealth.org/iiif/2/commonwealth:9880z8137/full/full/0/default.jpg
https://iiif.digitalcommonwealth.org/iiif/2/commonwealth:3f464n44x/full/full/0/default.jpg
https://iiif.digitalcommonwealth.org/iiif/2/commonwealth:76539h830/full/full/0/default.jpg
https://iiif.digitalcommonwealth.org/iiif/2/commonwealth:s1787416m/full/full/0/default.jpg
https://iiif.digitalcommonwealth.org/iiif/2/commonwealth:3f463853f/full/full/0/default.jpg
https://iiif.digitalcommonwealth.org/iiif/2/commonwealth:3f463851w/full/full/0/default.jpg
https://iiif.digitalcommonwealth.org/iiif/2/commonwealth:3f463833z/full/full/0/default.jpg

#### Presentation API

The Image API makes it easy to retrieve 

A IIIF manifest is the package that contains all the information related to a particular digital object, including the image itself as well as the metadata. The manifest is accessible via a URL that points to file which can be read by a IIIF tool or viewer, like Mirador. More about IIIF manifests.

IIIF Presentation manifests in JSON can be returned by appending `/manifest` to the URL for the item detail page. (A 404 will be returned if no manifest is available.)

